In [1]:
import pandas as pd
import json
from glob import glob
from tqdm import tqdm
import numpy as np

In [2]:
def read_trace_file(files, type, tqdm_desc):
    columns =['spdX', 'spdY', 'accX', 'accY', 'posX', 'posY', 'hedX', 'hedY', 'spdXNoise', 'spdYNoise', 'accXNoise', 'accYNoise', 'posXNoise', 'posYNoise', 'hedXNoise', 'hedYNoise', 'label', 'messageID']
    rows = []
    ret_df = pd.DataFrame()
    for file in tqdm(files, desc=tqdm_desc):
        with open(file) as f:
            for line in f:
                line = line.strip()
                dec = json.JSONDecoder()
                pos = 0
                while not pos == len(str(line)):
                    json_line, json_len = dec.raw_decode(str(line)[pos:])
                    pos += json_len

                    # json_line = json.loads(j)

                    if json_line['type'] == type:
                        label = 0 if 'A0' in file else 1
                        new_row = (
                            json_line['spd'][0],
                            json_line['spd'][1],
                            json_line['acl'][0],
                            json_line['acl'][1],
                            json_line['pos'][0],
                            json_line['pos'][1],
                            json_line['hed'][0],
                            json_line['hed'][1],

                            json_line['spd_noise'][0],
                            json_line['spd_noise'][1],
                            json_line['acl_noise'][0],
                            json_line['acl_noise'][1],
                            json_line['pos_noise'][0],
                            json_line['pos_noise'][1],
                            json_line['hed_noise'][0],
                            json_line['hed_noise'][1],


                            label, 
                            json_line['messageID'] if 'messageID' in json_line.keys() else -1
                        )
                        df = pd.DataFrame([list(new_row)], columns=columns)
                        rows.append(df)
    all_df = [ret_df]
    all_df.extend(rows)
    ret_df = pd.concat(all_df)
    return ret_df
                        # pd.concat(ret_df, df, ignore_index=True)

In [3]:
train_path = './data/train/*.*'
train_files = glob(train_path)
train_df = read_trace_file(train_files, 2, 'Aggregating Training Data')

test_path = './data/test/*.*'
test_files = glob(test_path)
test_df = read_trace_file(test_files, 3, 'Aggregating Testing Data')

Aggregating Testing Data: 100%|████████████████████████████████████████████████████████████████████████| 4369/4369 [08:32<00:00,  8.52it/s]


In [13]:
aggregation_functions = {'spdX': 'first', 'spdY': 'first', 'accX': 'first', 'accY': 'first', 'posX': 'first', 'posY': 'first', 'hedX': 'first', 'hedY': 'first', 'spdXNoise': 'first', 'spdYNoise': 'first', 'accXNoise': 'first', 'accYNoise': 'first', 'posXNoise': 'first', 'posYNoise': 'first', 'hedXNoise': 'first', 'hedYNoise': 'first', 'label': 'first', 'messageID': 'first', 'messageID': 'first'}
test_df = test_df.groupby(test_df['messageID']).aggregate(aggregation_functions)
test_df.reset_index(drop=True, inplace=True)

test_df = test_df.sort_values(by='messageID')
test_df.head()

,spdX,spdY,accX,accY,posX,posY,hedX,hedY,spdXNoise,spdYNoise,accXNoise,accYNoise,posXNoise,posYNoise,hedXNoise,hedYNoise,label,messageID
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,139624065
1,-0.161815,1.565888,-0.273165,2.643934,267.231275,31.157523,-0.102765,0.994706,-0.000000,-0.000000,0.002353,0.002353,3.658914,3.764030,0.166695,0.140246,1,139624871
2,-0.294866,-0.052932,-2.948658,-0.529318,1215.582709,975.080935,-0.984267,-0.176689,-0.000000,-0.000000,0.000564,0.000564,3.902924,3.904656,14.671405,14.320713,1,139635306
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,139636223
4,-5.695461,-12.187200,-0.732630,-1.567887,1370.555791,1102.278070,-0.458538,-0.888675,-0.011792,-0.027047,0.002017,0.004496,4.202668,4.186059,30.966393,59.268406,1,139636569


In [17]:
train_df.to_pickle('train.parquet')
test_df.to_pickle('./test_df.parquet')

In [18]:
X = train_df.loc[:, :'hedYNoise'].to_numpy()
Y = train_df.loc[:, 'label'].to_numpy()

X_test = test_df.loc[:, :'hedYNoise'].to_numpy()
Y_test = test_df.loc[:, 'label'].to_numpy()
message_ids = test_df.loc[:, 'messageID'].to_numpy()

# NN

In [19]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

from torch import nn
from torch.nn import functional as F


In [20]:
class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_shape,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
    return x

In [21]:
class dataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.length = self.x.shape[0]
 
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    def __len__(self):
        return self.length

In [22]:
def train(epochs, model, loss_fn, trainloader, optimizer, x, y):
    #forward loop
    losses = []
    accur = []
    
    print('Starting Training')
    
    for i in range(epochs):
        for j,(x_train,y_train) in enumerate(trainloader):
            
            #calculate output
            output = model(x_train)
        
            #calculate loss
            loss = loss_fn(output,y_train.reshape(-1,1))
        
            #accuracy
            predicted = model(torch.tensor(x,dtype=torch.float32))
            acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
            #backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i%50 == 0:
            losses.append(loss)
            accur.append(acc)
            print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))
    return losses, accur

In [23]:
def test(model, testloader, x_test):
    #forward loop
    preds = []
    for j,(x_test, y_test) in enumerate(testloader):
        preds = model(torch.tensor(x_test,dtype=torch.float32))
        preds.reshape(-1).detach().numpy().round()

    return preds

In [24]:
# Scaler Shit

sc = StandardScaler()
X = sc.fit_transform(X)

In [25]:
train_set = dataset(X, Y)
test_set = dataset(X_test, Y_test)

trainloader = DataLoader(train_set,batch_size=64,shuffle=False)
testloader = DataLoader(train_set,batch_size=test_set.length,shuffle=False)


In [ ]:
#hyper parameters
learning_rate = 0.01
epochs = 700
# Model , Optimizer, Loss
model = Net(input_shape=X.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

losses, accur = train(epochs, model, loss_fn, trainloader, optimizer, X, Y)

Starting Training


In [ ]:
#plotting the loss
plt.subplot(1,2,1)
plt.plot(losses)
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('loss')

#printing the accuracy
plt.subplot(1,2,2)
plt.plot(accur)
plt.title('Accuracy vs Epochs')
plt.xlabel('Accuracy')
plt.ylabel('loss')

In [ ]:
preds = test(model, testloader, X_test)
np.sum(preds) / len(preds)
results = pd.DataFrame(columns=['data_id', 'prediction'])
results['data_id'] = list(message_ids)
results['prediction'] = list(preds)
results.to_csv('results13.csv', index=False)